In [2]:
from typing import List

import numpy as np
from ioh import get_problem, logger, ProblemClass
from GA import s4018907_s4168216_GA, create_problem

In [3]:
from typing import Tuple 
import numpy as np
import ioh
from ioh import get_problem, logger, ProblemClass

def n_crossover(p1, p2, size, n=2, crossover_rate = 0.5):

    if np.random.rand() > crossover_rate:
        # If not, just return the parents as children (no crossover)
        return p1, p2

    split_positions = sorted(np.random.choice(range(size), n, replace=False))
    c1 = []
    c2 = []
    subarrays_p1 = np.split(p1, split_positions)
    subarrays_p2 = np.split(p2, split_positions)
    
    for i in range(len(subarrays_p1)):
        if (i % 2 == 0):
            for j in range(len(subarrays_p1[i])):
                c1.append(subarrays_p1[i][j])
                c2.append(subarrays_p2[i][j])
        else: 
            for j in range(len(subarrays_p2[i])):
                c1.append(subarrays_p2[i][j])
                c2.append(subarrays_p1[i][j])
    return c1,c2

def mating_selection(population, pop_fitness):
    f_min = min(pop_fitness)
    f_sum = sum(pop_fitness) - (f_min - 0.001) * len(pop_fitness)
    
    rw = [(pop_fitness[0] - f_min + 0.001)/f_sum]
    for i in range(1,len(pop_fitness)):
        rw.append(rw[i-1] + (pop_fitness[i] - f_min + 0.001) / f_sum)
    
    select_parent = []
    for i in range(2) :
        r = np.random.uniform(0,1)
        index = 0
        # print(rw,r)
        while(r > rw[index]) :
            index = index + 1
        
        select_parent.append(population[index].copy())
    return select_parent

def mutate(c, mutation_rate):
    ind_length = len(c)
    for j in range(ind_length):  
        if np.random.uniform(0, 1) < mutation_rate:
            swap_idx = np.random.randint(0, ind_length)
            c[j], c[swap_idx] = c[swap_idx], c[j]  # Swap mutation

    return c

def s4018907_s4168216_GA(problem: ioh.problem.PBO, init_pop_size: int, mutation_rate: float, crossover_rate: float, budget: int) -> None:
    # initial_pop = ... make sure you randomly create the first population
    #initial_pop_size = pop_size
    # mutation_rate = 
    #crossover_rate = 0.5
    population = []
    pop_fitness = []

    for i in range(init_pop_size):
        # Initialization
        population.append(np.random.randint(2, size = problem.meta_data.n_variables))
        pop_fitness.append(problem(population[i]))

    # `problem.state.evaluations` counts the number of function evaluation automatically,
    # which is incremented by 1 whenever you call `problem(x)`.
    # You could also maintain a counter of function evaluations if you prefer.
    while problem.state.evaluations < budget:
        parents = mating_selection(population, pop_fitness)
        p1 = parents[0]
        p2 = parents[1]
        c1, c2 = n_crossover(p1, p2, problem.meta_data.n_variables, crossover_rate = crossover_rate)
        mutated_c1 = mutate(c1, mutation_rate)
        mutated_c2 = mutate(c2, mutation_rate)
        f1 = problem(mutated_c1)
        f2 = problem(mutated_c2)
        #print("f1", f1)
        #print("f2", f2)
        population.append(mutated_c1)
        population.append(mutated_c2)
    print("F1: ", f1)
    print("F2: ", f2)
    return f1, f2 


def create_problem(dimension: int, fid: int, name:str) -> Tuple[ioh.problem.PBO, ioh.logger.Analyzer]:
    # Declaration of problems to be tested.
    problem = get_problem(fid, dimension=dimension, instance=1, problem_class=ProblemClass.PBO)
    print("creating problem")
    # Create default logger compatible with IOHanalyzer
    # `root` indicates where the output files are stored.
    # `folder_name` is the name of the folder containing all output. You should compress the folder 'run' and upload it to IOHanalyzer.
    l = logger.Analyzer(
        root="data",  # the working directory in which a folder named `folder_name` (the next argument) will be created to store data
        folder_name="run",  # the folder name to which the raw performance data will be stored
        algorithm_name= name,  # name of your algorithm
        algorithm_info="Practical assignment of the EA course",
    )
    # attach the logger to the problem
    problem.attach_logger(l)
    return problem, l



In [4]:
budget = 5000000

# To make your results reproducible (not required by the assignment), you could set the random seed by
# `np.random.seed(some integer, e.g., 42)`

# Hyperparameters to tune, e.g.
np.random.seed(42)
hyperparameter_space = {
    "population_size": [50, 100, 200, 500, 1000],
    "mutation_rate": [0.01, 0.05, 0.1],
    "crossover_rate": [0.6, 0.8]
}


In [6]:
# Fixed Hyperparameter tuning function with the positional argument issue resolved
def tune_hyperparameters() -> List:
    # You should decide/engineer the `score` yourself, which is the tuning objective
    best_score = float('-inf')
    best_params = None
    # create the LABS problem and the data logger


    for pop_size in hyperparameter_space['population_size']:
        for mutation_rate in hyperparameter_space['mutation_rate']:
            for crossover_rate in hyperparameter_space['crossover_rate']:

                F18, _logger18 = create_problem(dimension=50, fid=18, name=f"pop_size={pop_size}_mr={mutation_rate}_cr={crossover_rate}")
                # create the N-Queens problem and the data logge
                F23, _logger23 = create_problem(dimension=49, fid=23, name=f"pop_size={pop_size}_mr={mutation_rate}_cr={crossover_rate}")

                
                print("running hyperparameter tuning for ", pop_size, mutation_rate, crossover_rate)

                print("Running GA on F18")
                score_f18_1, score_f18_2 = s4018907_s4168216_GA(F18, pop_size, mutation_rate, crossover_rate, budget)
                score_f18 = np.mean([score_f18_1, score_f18_2])
                print("score for F18: ", score_f18)

                F18.reset()

                if score_f18 > best_score:
                    best_score = score_f18
                    print("best score for F18: ", best_score)
                    best_params = [pop_size, mutation_rate, crossover_rate]

                print("Running GA on F23")
                score_f23_1, score_f23_2 = s4018907_s4168216_GA(F23, pop_size, mutation_rate, crossover_rate, budget)
                score_f23 = np.mean([score_f23_1, score_f23_2])
                print("score for F23: ", score_f23)

                F23.reset()

                if score_f23 > best_score:
                    best_score = score_f23
                    print("best score for F23: ", best_score)
                    best_params = [pop_size, mutation_rate, crossover_rate]

    return best_params

In [7]:
    '''
        for mutation_rate in hyperparameter_space['mutation_rate']:
        for crossover_rate in hyperparameter_space['crossover_rate']:
    '''

"\n    for mutation_rate in hyperparameter_space['mutation_rate']:\n    for crossover_rate in hyperparameter_space['crossover_rate']:\n"

In [8]:
# The error occurs because the function 'tune_hyperparameters' is returning a single object (likely a float or another value),
# while the code expects it to return three unpackable values (population_size, mutation_rate, crossover_rate).

# Fix: Ensure that the 'tune_hyperparameters' function returns a tuple with 3 values or change the calling code to match
# the actual return value of the function.

if __name__ == "__main__":

    # Call the hyperparameter tuning function
    best_params = tune_hyperparameters()
    #if len(best_params) == 3:
    population_size, mutation_rate, crossover_rate = best_params
    print(population_size)
    print(mutation_rate)
    print(crossover_rate)


creating problem
creating problem
running hyperparameter tuning for  50 0.01 0.6
Running GA on F18
F1:  0.8350033400133601
F2:  1.6960651289009498
score for F18:  1.265534234457155
best score for F18:  1.265534234457155
Running GA on F23
F1:  -457.0
F2:  -254.0
score for F23:  -355.5
creating problem
creating problem
running hyperparameter tuning for  50 0.01 0.8
Running GA on F18
F1:  1.0548523206751055
F2:  0.9881422924901185
score for F18:  1.021497306582612
Running GA on F23
F1:  -336.0
F2:  -356.0
score for F23:  -346.0
creating problem
creating problem
running hyperparameter tuning for  50 0.05 0.6
Running GA on F18
F1:  2.1079258010118043
F2:  1.2388503468780971
score for F18:  1.6733880739449507
best score for F18:  1.6733880739449507
Running GA on F23
F1:  -423.0
F2:  -389.0
score for F23:  -406.0
creating problem
creating problem
running hyperparameter tuning for  50 0.05 0.8
Running GA on F18
F1:  1.4863258026159334
F2:  1.2388503468780971
score for F18:  1.3625880747470154


In [1]:
print(best_params)

NameError: name 'best_params' is not defined

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6809d180-5347-4c49-92ff-04f56374e52e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>